In [5]:
import urllib as u
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

api_key='64d94994afb9689c49b660ebe9dee39c'

In [6]:
# Function to read in json data from FMP
def get_json_data(url):
    '''
    Scrape data (which must be json format) from given url
    Input: url to financialmodelingprep API
    Output: json file
    '''
    response = urlopen(url)
    dat = response.read().decode('utf-8')
    return json.loads(dat)

In [3]:
# Function to get sector of a Stock from its profile json file
def find_sector(json):
    
    for k, v in a.items():
        if k == 'symbol':
            symbol=v

    for k, v in a.items():
        if k == 'profile':
            profile=v

    for k, v in profile.items():
        if k == 'sector':
            sector=v
    return(symbol, sector)

In [13]:
# Function to get Year 2019 Financial Indicators from json file
def find_fin_indicators(json, indicator):
    cat = ['financials','growth']
    
    for k, v in json.items():
        if k == 'symbol':
            symbol=v

    for k, v in json.items():
        if k in cat:
            category = v

    indicators_2019 = [[(k,v) for k,v in d.items()] for d in category if d['date'][:4] == '2019'] # Generator creates a list of lists
    indicators_2019_list = [item for sublist in indicators_2019 for item in sublist] # Creating a flat list

    a = []

    for tup in (x for x in indicators_2019_list if x[0] in indicator):
        a.append((symbol,) + tup)
    return a

In [94]:
# Function to get Year 2019 Ratios from json file
def find_ratios_indicators(json, indicator):
    global symbol, ratios
    
    for k in json.items():
        if k == 'symbol':
            symbol = k
    
    for k in json.items():
        if k == 'ratios':
            ratios = k
            
    ratios_2019 = [d for d in ratios if d['date'][:4] == '2019']
    
    unpack1 = ratios_2019[0]['investmentValuationRatios']
    unpack2 = ratios_2019[0]['profitabilityIndicatorRatios']
    unpack3 = ratios_2019[0]['operatingPerformanceRatios']
    unpack4 = ratios_2019[0]['liquidityMeasurementRatios']
    unpack5 = ratios_2019[0]['debtRatios']
    unpack6 = ratios_2019[0]['cashFlowIndicatorRatios']
    
    unpack = unpack1
    unpack.update(unpack2)
    unpack.update(unpack3)
    unpack.update(unpack4)
    unpack.update(unpack5)
    unpack.update(unpack6)
    
    full_set = []
    reduced_set =[]

    for row in unpack.items():
            full_set.append(row)
    
    for tup in (x for x in full_set if x[0] in indicator):
        reduced_set.append((symbol,) +tup)
    
    return reduced_set

In [ ]:
# Full list of available stocks from FMP
url = 'https://financialmodelingprep.com/api/v3/stock/list?apikey=' + api_key
ticks_json = get_json_data(url)

stocks = pd.DataFrame(ticks_json)
stocks = stocks[['symbol','name','exchange']]
stocks = stocks.dropna(how='any',subset=['symbol','name','exchange'])  

In [ ]:
# Limit the analysis to stocks of the New York Stock Exchange
stocks_nyse = stocks[(stocks['exchange'] == 'New York Stock Exchange')]
stocks_nyse = stocks_nyse.reset_index()
tickers_nyse = stocks_nyse['symbol'].tolist()

In [ ]:
# Get sectors from company profiles
tickers_nyse_sector = []

for tick in tqdm(tickers_nyse):
    url = 'https://financialmodelingprep.com/api/v3/company/profile/' + tick + '?apikey=' + api_key 
    a = get_json_data(url)
    tickers_nyse_sector.append(find_sector(a))

In [ ]:
tickers_nyse_sector = pd.DataFrame(tickers_nyse_sector)
tickers_nyse_sector.columns = ['symbol', 'sector']

stocks_nyse = stocks_nyse.merge(tickers_nyse_sector)

del stocks_nyse['index','exchange']

In [4]:
# Load financial indicators from the provided .txt file
indicators = []
filename = 'indicators.txt'
with open(filename, 'r') as f:
    for line in f:
        indicators.append(line.strip('\n'))

In [14]:
# Get Financials Indicators
inc_statement_indicators = []
cash_flow_indicators = []
bs_indicators = []
fs_growth_indicators = []

for tick in tqdm(tickers_nyse):
    try:
        url0 = 'https://financialmodelingprep.com/api/v3/financials/income-statement/' + tick + '?apikey=' + api_key
        url1 = 'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/' + tick + '?apikey=' + api_key
        url2 = 'https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/' + tick + '?apikey=' + api_key
        url3 = 'https://financialmodelingprep.com/api/v3/financial-statement-growth/' + tick + '?apikey=' + api_key

        a0 = get_json_data(url0)
        a1 = get_json_data(url1)
        a2 = get_json_data(url2)
        a3 = get_json_data(url3)
    except:
        pass
    
    inc_statement_indicators.append(find_fin_indicators(a0,indicators))
    cash_flow_indicators.append(find_fin_indicators(a1,indicators))
    bs_indicators.append(find_fin_indicators(a2,indicators))
    fs_growth_indicators.append(find_fin_indicators(a3,indicators))
    
    fin_indicators = inc_statement_indicators + cash_flow_indicators + bs_indicators + fs_growth_indicators
    

100%|██████████| 2/2 [00:06<00:00,  3.10s/it]
